In [ ]:
import os
import pandas as pd
from groq import Groq

# Initialize the Groq client
client = Groq(
    api_key="API_KEY"
)

# Function to call Groq API with LLAMA 3 model
def call_groq_api(prompt, model="llama3-70b-8192"):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model=model,
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error calling Groq API: {e}")
        return None

# Function to generate user prompts based on incorrect answers
def generate_prompts_from_csv(df):
    user_prompts = {}

    # Load the CSV files
    quiz_df = pd.read_csv('Module1_Notes_to_Quiz/quiz.csv')
    response_df = pd.read_csv('Module4_Re-Quiz_Analysis/Test_Responce.csv')

    # Iterate through each student's responses
    for index, row in response_df.iterrows():
        enrollment_no = row['Enrollment No']
        incorrect_questions = []

        # Compare each response with the correct answer
        for i, question in enumerate(quiz_df['Question']):
            correct_answer = quiz_df['Correct Answer'][i]
            student_response = row[i + 1]  # Responses start from the second column

            if student_response != correct_answer:
                incorrect_questions.append(question)

        # Generate prompt for the current student only
        prompt = (
            "You are a skilled educational writer, known for explaining complex topics in a way that's easy to understand. "
            "Based on the following questions, generate a detailed, insightful article that balances depth with clarity.\n\n"

            "## Article Requirements:\n"
            "- Minimum of 1000 words, focused on providing clear, well-researched, and accurate information.\n"
            "- Maintain a conversational yet professional tone that keeps the reader engaged.\n"
            "- Explain each topic thoroughly, using simple language while avoiding jargon.\n"
            "- Organize the article with subheadings to improve readability and flow.\n"
            "- Structure the content with an introduction, body sections, and a conclusion.\n"
            "- Use real-world examples or case studies to make concepts relatable and practical.\n"
            "- Ensure the article is both educational and enjoyable to read, helping the reader grasp even complex ideas effortlessly.\n"
            "- Generate the response in HTML format, ensuring proper use of tags like <h1>, <h2>, <p>, <ul>, <li>, and <strong> where appropriate.\n\n"

            "## Article Format (in HTML):\n"
            "## Article Format (in HTML with Tailwind CSS):\n"
            "<!DOCTYPE html>\n"
            "<html>\n"
            "<head>\n"
            "<title>Generated Title Based on Questions</title>\n"
            "<link href=\"https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css\" rel=\"stylesheet\">\n"
            "</head>\n"
            "<body class=\"bg-gray-50 text-gray-700 font-sans\">\n\n"

            "<div class=\"max-w-4xl mx-auto p-8\">\n"

            "<h1 class=\"text-4xl font-bold text-gray-900 mb-6\">Generated Title Based on Questions</h1>\n\n"

            "<h2 class=\"text-2xl font-semibold text-gray-800 mt-8 mb-4\">Introduction</h2>\n"
            "<p class=\"text-gray-600 mb-4\">Begin with a captivating introduction that highlights the importance of the topic in relation to the questions. "
            "Provide a clear overview of what the reader will learn and why it matters.</p>\n\n"

            "<h2 class=\"text-2xl font-semibold text-gray-800 mt-8 mb-4\">Section 1: Core Concepts Explained</h2>\n"
            "<p class=\"text-gray-600 mb-4\">Introduce the key concepts related to the questions. Provide simple, clear definitions and offer the necessary "
            "background to set the stage for deeper exploration.</p>\n\n"

            "<h2 class=\"text-2xl font-semibold text-gray-800 mt-8 mb-4\">Section 2: In-Depth Answers to Each Question</h2>\n"
            "<p class=\"text-gray-600 mb-4\">Answer each question comprehensively, breaking down the topic into easy-to-understand parts. Use lists, "
            "bullet points, or examples where appropriate to clarify complex points.</p>\n"
            "<ul class=\"list-disc pl-6 text-gray-600 mb-4\">\n"
            "  <li>Example point 1</li>\n"
            "  <li>Example point 2</li>\n"
            "</ul>\n\n"
        )

        # Add only incorrect answers for the current student to the prompt
        for question in incorrect_questions:
            prompt += f"<li>{question}</li>\n"
        
        prompt += "</ul>\n\n"

        prompt += (
           "<h2 class=\"text-2xl font-semibold text-gray-800 mt-8 mb-4\">Section 3: Real-World Applications and Examples</h2>\n"
            "<p class=\"text-gray-600 mb-4\">Demonstrate how these concepts apply in the real world. Use relevant case studies, personal stories, or examples to "
            "highlight the importance of the topics in practice.</p>\n\n"

            "<h2 class=\"text-2xl font-semibold text-gray-800 mt-8 mb-4\">Section 4: Addressing Common Challenges</h2>\n"
            "<p class=\"text-gray-600 mb-4\">Discuss any challenges, misconceptions, or mistakes that readers might encounter with these topics. "
            "Provide practical solutions or strategies to help them avoid or overcome these issues.</p>\n\n"

            "<h2 class=\"text-2xl font-semibold text-gray-800 mt-8 mb-4\">Conclusion</h2>\n"
            "<p class=\"text-gray-600 mb-4\">Summarize the key takeaways from the article, reinforcing the main insights. End with a thoughtful conclusion that "
            "encourages readers to explore the topic further and apply what they've learned.</p>\n\n"

            "</div>\n"
            "</body>\n"
            "</html>\n\n"
        )

        user_prompts[enrollment_no] = prompt

    return user_prompts

# Function to create an article for each user based on incorrect answers
def create_articles(user_prompts, output_folder="Module5_Re-Quiz_Custom_Notes/generated_articles"):
    os.makedirs(output_folder, exist_ok=True)

    for reg_no, prompt in user_prompts.items():
        print(f"Generating article for user {reg_no}...")

        generated_text = call_groq_api(prompt)

        if generated_text:
            file_name = os.path.join(output_folder, f"user_{reg_no}_article.html")
            with open(file_name, "w", encoding="utf-8") as file:
                file.write(generated_text)
            print(f"Article saved for user {reg_no} as {file_name}\n")
        else:
            print(f"Failed to generate article for user {reg_no}\n")

# Main function to run the script
def main(csv_file_path):
    try:
        df = pd.read_csv(csv_file_path)
        user_prompts = generate_prompts_from_csv(df)
        create_articles(user_prompts)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    csv_file_path = "Module4_Re-Quiz_Analysis/Test_Responce.csv" 
    main(csv_file_path)